In [2]:
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense
import operator
import numpy as np
import os
from PIL import Image
import glob

import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

import cv2
import math 

ModuleNotFoundError: No module named 'matplotlib'

In [2]:
#main emotions are dataset comes with
emotions = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]

#load data into our notebook
def load_emotions_data(path):
    x = []
    y = []
    for i, emotion in enumerate(emotions):
        print(f"\r  {path}/{emotion}... ({i+1}/{len(emotions)})          ", end='')
        for filename in glob.glob(f"{path}/{emotion}/*.jpg"):
            img = Image.open(filename)
            img = np.array(img.getdata()).reshape(48, 48) / 255
            x.append(img)
            y_onehot = np.zeros(len(emotions))
            y_onehot[i] = 1
            y.append(y_onehot)
    print(f"\r  loaded {path}.                      ")
    return np.array(x), np.array(y)

print("Loading image data...")
x_train, y_train = load_emotions_data("data/train")
x_test, y_test = load_emotions_data("data/test")


Loading image data...
  loaded data/train.                      
  loaded data/test.                      


#OUR MODEL
batch_size = 128
input_shape = (48, 48, 1)

model = Sequential()
model.add(Conv2D(32, kernel_size=5, activation='relu', input_shape=input_shape))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=5, activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=5, activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='tanh'))
model.add(Dense(len(emotions), activation='softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

#train model
model.fit(x_train, y_train, batch_size=batch_size, epochs=1)

In [3]:
#model = tf.keras.models.load_model('saved_model/saved_model.pb')
batch_size = 128
input_shape = (48, 48, 1)
model = tf.keras.models.load_model("saved_model")

2022-05-03 15:19:18.653907: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
#display confusion matrix to demonstrate accuracy
def plot_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred, normalize='true')
    fig = plt.figure(figsize=(6,6))
    matplotlib.rcParams.update({'font.size': 16})
    ax  = fig.add_subplot(111)
    matrix = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.RdYlGn)
    # fig.colorbar(matrix) 
    for i in range(0,len(emotions)):
        for j in range(0,len(emotions)):  
            ax.text(j,i,np.round(cm[i,j],2),va='center', ha='center')
    ticks = np.arange(len(emotions))
    ax.set_xticks(ticks)
    ax.set_xticklabels(emotions, rotation=45)
    ax.set_yticks(ticks)
    ax.set_yticklabels(emotions)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

_ = model.evaluate(x_test, y_test, batch_size=batch_size)

y_prob = model.predict(x_test, batch_size=32, verbose=0)
y_pred = [np.argmax(prob) for prob in y_prob]
y_true = [np.argmax(true) for true in y_test]

plot_confusion_matrix(y_true, y_pred)
plt.show()

29/57 [==============>...............] - ETA: 0s - loss: 1.0611 - accuracy: 0.5814

KeyboardInterrupt: 

In [6]:
#dictonary which saves the basic emojis and allows for easy mapping
emojiDict = {'happy': "\U0001F603",
             'surprise':"\U0001F62E",
             'angry':"\U0001F621",
             'neutral': "\U0001F610",
             'disgust':"\U0001F922",
             'fear':"\U0001F631",
             "sad":"\U0001F625"}
emojiDict

{'happy': '😃',
 'surprise': '😮',
 'angry': '😡',
 'neutral': '😐',
 'disgust': '🤢',
 'fear': '😱',
 'sad': '😥'}

In [12]:
#use open cv to get real time camera/image data

cv2.namedWindow("preview")
vc = cv2.VideoCapture(0)

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

if vc.isOpened(): # try to get the first frame
    rval, frame = vc.read()
else:
    rval = False

while rval:
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    
    #get cropped image so it focuses on face
    faces = face_cascade.detectMultiScale(frame, 1.1, 8)
    if(len(faces) > 0):
        (x, y, w, h) = faces[0]
    else:
        x, y, w, h = 0, 0, 50, 1
    if(w > h):
        w = h
    else:
        h = w
    cropped_image = frame[y:y+h, x:x+w]
    
    #makes sure image is in correct format for model... reformatting
    img = Image.fromarray(cropped_image , 'L')
    img = img.resize((48, 48))
    img = np.array(img.getdata())
    img = img.reshape(1, 48, 48, 1)
    img = img/255
    
    #run model and get probabilities for emoji
    y_probs = model(img).numpy().flatten()
    y_preds = np.flip(np.argsort(y_probs)).flatten()
    y_preds = [(emojiDict[emotions[y_preds[i]]], np.round(y_probs.item(y_preds[i]), 2)) for i in range(len(emotions))]

    print(f"\r{y_preds}".ljust(150), end="")
    #print(y_probs)

    cv2.imshow("preview", cropped_image)
    rval, frame = vc.read()
    key = cv2.waitKey(20)
    if key == 27: # exit on ESC
        break


vc.release()
cv2.destroyWindow("preview")
cv2.waitKey(1)


[('😃', 0.94), ('😡', 0.02), ('😐', 0.02), ('😥', 0.02), ('😱', 0.01), ('😮', 0.0), ('🤢', 0.0)]                                                            

KeyboardInterrupt: 

In [8]:
#makes a dictionary for each emoji and their given values for each of the 7 emotions
#does this by using model to analyze pre matched photos to their given emoji
import os
facePredict = {}
directory = 'faces'
for filename in os.listdir(directory):
    if filename[-3:]!="ore":
        f = os.path.join(directory, filename)
        
        img = Image.open(f)
        img = np.array(img.getdata()).reshape(48, 48) / 255
        img = np.expand_dims(img,0)
        y_probs = model(img).numpy().flatten()
        y_predss = np.flip(np.argsort(y_probs)).flatten()
        y_predss = [(emojiDict[emotions[y_predss[i]]], np.round(y_probs.item(y_predss[i]), 2)) for i in range(len(emotions))]
    
        facePredict[filename[:-4]] =y_predss
       
facePredict

{'🥺': [('😥', 0.82),
  ('😱', 0.1),
  ('😐', 0.05),
  ('😡', 0.02),
  ('😃', 0.01),
  ('🤢', 0.0),
  ('😮', 0.0)],
 '😟': [('😥', 0.38),
  ('😱', 0.28),
  ('😐', 0.23),
  ('😡', 0.1),
  ('😮', 0.01),
  ('😃', 0.01),
  ('🤢', 0.0)],
 '😊': [('😃', 0.95),
  ('😐', 0.02),
  ('😮', 0.01),
  ('😥', 0.01),
  ('😱', 0.0),
  ('😡', 0.0),
  ('🤢', 0.0)],
 '😭': [('😥', 0.33),
  ('😱', 0.25),
  ('😃', 0.13),
  ('😐', 0.12),
  ('😡', 0.1),
  ('😮', 0.05),
  ('🤢', 0.03)],
 '😮': [('😱', 0.42),
  ('😡', 0.34),
  ('🤢', 0.1),
  ('😥', 0.06),
  ('😃', 0.05),
  ('😮', 0.02),
  ('😐', 0.01)],
 '😯': [('😮', 0.69),
  ('😱', 0.29),
  ('😡', 0.01),
  ('😥', 0.01),
  ('😃', 0.0),
  ('😐', 0.0),
  ('🤢', 0.0)],
 '😫': [('😥', 0.42),
  ('😡', 0.24),
  ('😃', 0.2),
  ('😱', 0.1),
  ('😐', 0.02),
  ('😮', 0.01),
  ('🤢', 0.01)],
 '😨': [('😥', 0.54),
  ('😐', 0.19),
  ('😱', 0.17),
  ('😡', 0.08),
  ('😃', 0.01),
  ('😮', 0.01),
  ('🤢', 0.0)],
 '😤': [('😡', 0.67),
  ('😥', 0.15),
  ('😱', 0.07),
  ('🤢', 0.05),
  ('😐', 0.04),
  ('😃', 0.01),
  ('😮', 0.0)],
 '😳': [('😱', 0.52)

In [11]:
#calculates how close the expression is to an emoji
def dist(one, two):
    l = ['😥','😱','😐','😡','😃','🤢','😮']
    value = 0
    for i in range(7):
        val = 0
        
        if(one[i][0] == l[i]):
            val+=one[i][1]
        if(two[i][0] == l[i]):
            val-=two[i][1]
        val = val*val
        value+=val
    return math.sqrt(value)           
#print(y_preds)
#print(facePredict['😐'])
dist(y_preds,facePredict['😐'])


NameError: name 'y_preds' is not defined

In [10]:
#goes through each emoji and sees how close it is... 
#returns the 3 most likely emoji to match expression
dictt = {}
for item in facePredict.items():
    distt = dist(y_preds, item[1])
    dictt[item[0]]= distt
  
sorted_dicdata = sorted(dictt.items(), key=operator.itemgetter(1),    reverse=True)
    

sorted_dicdata[:3]

NameError: name 'dist' is not defined

In [13]:
#FINAL.... RETURNS 3 EMOJIS REAL TIME

cv2.namedWindow("preview")
vc = cv2.VideoCapture(0)

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

if vc.isOpened(): # try to get the first frame
    rval, frame = vc.read()
else:
    rval = False

while rval:
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    
    #get cropped image so it focuses on face
    faces = face_cascade.detectMultiScale(frame, 1.1, 8)
    if(len(faces) > 0):
        (x, y, w, h) = faces[0]
    else:
        x, y, w, h = 0, 0, 50, 1
    if(w > h):
        w = h
    else:
        h = w
    cropped_image = frame[y:y+h, x:x+w]
    
    #makes sure image is in correct format for model... reformatting
    img = Image.fromarray(cropped_image , 'L')
    img = img.resize((48, 48))
    img = np.array(img.getdata())
    img = img.reshape(1, 48, 48, 1)
    img = img/255
    
    #run model and get probabilities for emoji
    y_probs = model(img).numpy().flatten()
    y_pred = np.flip(np.argsort(y_probs)).flatten()
    y_pred = [(emojiDict[emotions[y_pred[i]]], np.round(y_probs.item(y_pred[i]), 2)) for i in range(len(emotions))]

    #print(f"\r{y_preds}".ljust(150), end="")
    dicttt = {}
    for item in facePredict.items():
        disttt = dist(y_pred, item[1])
        dicttt[item[0]]= disttt
  
    sorted_dic = sorted(dicttt.items(), key=operator.itemgetter(1),    reverse=True)
 
    print(f"\r{sorted_dic[-3:]}".ljust(150), end="")

   

    cv2.imshow("preview", cropped_image)
    rval, frame = vc.read()
    key = cv2.waitKey(20)
    if key == 27: # exit on ESC
        break


vc.release()
cv2.destroyWindow("preview")
cv2.waitKey(1)


[('😁', 0.02), ('🙂', 0.02), ('😃', 0.02)]                                                                                                              

KeyboardInterrupt: 